In [3]:
from pennylane import numpy as np
import pennylane as qml
import openfermion

In [4]:
symbols = ["N", "N"]
dist = 2.0741632451/2 # in AU 
coordinates = np.array([0.0, 0.0, -dist, 0.0, 0.0, dist ])

In [5]:
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  20
The Hamiltonian is    (-66.19208148039114) [I0]
+ (1.0090899129344706) [Z18]
+ (1.0090899129344706) [Z19]
+ (1.1672982708162567) [Z15]
+ (1.1672982708162576) [Z14]
+ (1.1672982708164341) [Z17]
+ (1.1672982708164346) [Z16]
+ (1.2839292165168426) [Z12]
+ (1.2839292165168426) [Z13]
+ (1.337278770491843) [Z6]
+ (1.337278770491843) [Z7]
+ (1.3441573689457134) [Z10]
+ (1.3441573689457134) [Z11]
+ (1.3441573689458224) [Z8]
+ (1.3441573689458235) [Z9]
+ (1.8364679519730398) [Z4]
+ (1.8364679519730398) [Z5]
+ (9.607991062014253) [Z2]
+ (9.607991062014255) [Z3]
+ (9.608121952212219) [Z0]
+ (9.608121952212219) [Z1]
+ (-9.777319842059924e-10) [Y6 Y8]
+ (-9.777319842059924e-10) [X6 X8]
+ (-1.204269631127393e-10) [Y0 Y2]
+ (-1.204269631127393e-10) [X0 X2]
+ (-3.5855892700698172e-12) [Y12 Y14]
+ (-3.5855892700698172e-12) [X12 X14]
+ (1.9457399074188286e-11) [Y2 Y4]
+ (1.9457399074188286e-11) [X2 X4]
+ (1.2044731759961687e-10) [Y1 Y3]
+ (1.2044731759961687e-10) [X1 X3]
+ (8.4281

In [4]:
dev = qml.device("default.qubit", wires=qubits)

In [5]:
electrons = 10
hf = qml.qchem.hf_state(electrons, qubits)
print(hf)

[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]


In [18]:
def circuit(param, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(param, wires=[0, 1, 10, 11,] )

In [19]:
@qml.qnode(dev, interface="autograd")
def cost_fn(param):
    circuit(param, wires=range(qubits))
    return qml.expval(H)

In [20]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)

In [21]:
theta = np.array(0.0, requires_grad=True)

In [22]:
energy = [cost_fn(theta)]

# store the values of the circuit parameter
angle = [theta]

max_iterations = 100
conv_tol = 1e-06

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy.append(cost_fn(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_energy)

    if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" f"Optimal value of the circuit parameter = {angle[-1]:.4f}")